# MATH&ML-13. Временные ряды. Часть II

## 4. Статистические модели прогнозирования. Практика

см. project-Part1.ipynb

## 8. Валидация временных рядов

In [6]:
# 8.1 1. Разбейте временной ряд на три набора (3 train + 3 test). 
# Для этого воспользуйтесь классом TimeSeriesSplit из sklearn.model_selection с параметрами n_splits=3 и test_size=7 
# (или max_train_size=df.sales.count()-7). Наборы данных можно получить, вызвав метод split (временной ряд) 
# у инициализированного TimeSeriesSplit.
import pandas as pd 
from sklearn.model_selection import TimeSeriesSplit

data = pd.read_csv('data/ghana_gdp.csv', parse_dates=["year"], index_col = ["year"])
tscv = TimeSeriesSplit(n_splits=3,test_size=7)
data_ts = tscv.split(data)
# 2. Выведите размеры полученных подвыборок:
train_test_groups = tscv.split(data) 
for train_index, test_index in train_test_groups:
    print("TRAIN size:", len(train_index), "TEST size:", len(test_index))

TRAIN size: 41 TEST size: 7
TRAIN size: 48 TEST size: 7
TRAIN size: 55 TEST size: 7


##  10. Другие методы предсказания временных рядов

#### PROPHET

In [18]:
import pandas as pd

from prophet import Prophet

url = 'https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv'

df = pd.read_csv(url)

df.head()

,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572


In [19]:
# Обучим модель на датасете:

model = Prophet()
model.fit(df)

23:03:08 - cmdstanpy - INFO - Chain [1] start processing
23:03:08 - cmdstanpy - INFO - Chain [1] done processing
23:03:08 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
23:03:08 - cmdstanpy - INFO - Chain [1] start processing
23:03:08 - cmdstanpy - INFO - Chain [1] done processing
23:03:08 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/prophet/stan_model/prophet_model.bin random seed=96300 data file=/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/tmpqprv5ytj/gme9wmxc.json init=/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/tmpqprv5ytj/o_9l45jz.json output file=/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/tmpqprv5ytj/prophet_modeluzxmmyca/prophet_model-20230302230308.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[77139]: Library not loaded: '@rpath/libtbb.dylib'
  Referenced from: '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/prophet/stan_model/prophet_model.bin'
  Reason: tried: '/private/var/folders/tp/mcxy8wd524b65_m2_4n493dh0000gn/T/pip-install-s4iklrnv/prophet_d587fc2dadbf48198c784125c26d078a/build/lib.macosx-10.9-universal2-cpython-310/prophet/stan_model/cmdstan-2.26.1/stan/lib/stan_math/lib/tbb/libtbb.dylib

In [12]:
future_df = model.make_future_dataframe(periods=365)

future_df.tail()

,ds
3265,2017-01-15
3266,2017-01-16
3267,2017-01-17
3268,2017-01-18
3269,2017-01-19


In [13]:
forecast_df = model.predict(future_df)
forecast_df[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

KeyError: 'k'

In [ ]:
# Мы можем отобразить прогноз, вызвав метод Prophet.plot и передав ему полученный датафрейм с результатами прогноза:
fig = model.plot(forecast_df)

In [ ]:
# Чтобы увидеть компоненты ряда, можно использовать метод Prophet.plot_components. 
# По умолчанию предоставляются тренд и годовая и недельная сезонность временного ряда.

fig = model.plot_components(forecast_df)

### NEURALPROPHET